In [90]:
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
import pickle
from tqdm.auto import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
import pyarrow.compute as pc
from scipy.sparse import coo_matrix, csr_matrix, hstack, vstack, save_npz
from rdkit.Chem import rdFingerprintGenerator

In [91]:
parquet_file = pq.ParquetFile('../../train.parquet')
ecfp_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

with open('../split_generation/block_splits.pkl', 'rb') as f:
    ((split1_block1, split2_block1, split3_block1, split4_block1, split5_block1), (split1_block23, split2_block23, split3_block23, split4_block23, split5_block23)) = pickle.load(f)

for x in tqdm(parquet_file.iter_batches(batch_size=100000)):
    x = x.to_pandas()
    x_neg = x[x['binds']==0].sample(frac=0.05, replace=False, random_state=1)
    x = pd.concat([x[x['binds']==1], x_neg]).drop_duplicates()
    # x_test x[x['buildingblock1_smiles'].isin(split1_block1) & x['buildingblock2_smiles'].isin(split1_block23) & x['buildingblock3_smiles'].isin(split1_block23)]
    mol = [Chem.MolFromSmiles(x.replace("Dy","C")) for x in x['molecule_smiles']]
    ecfp = [ecfp_gen.GetFingerprint(x).ToList() for x in mol]
    result = csr_matrix(ecfp)
    one_hot = np.zeros((len(x), 3))
    one_hot[x['protein_name']=="BRD4",0] = 1
    one_hot[x['protein_name']=="HSA",1] = 1
    one_hot[x['protein_name']=="sEH",2] = 1
    result = hstack([result, one_hot, np.expand_dims(x['binds'],1)])
    save_npz('test.npz', result)
    break

0it [00:00, ?it/s]

0it [00:02, ?it/s]


In [86]:
result

<510x2052 sparse matrix of type '<class 'numpy.float64'>'
	with 34015 stored elements in COOrdinate format>

In [83]:
x[x['buildingblock1_smiles'].isin(split1_block1) & x['buildingblock2_smiles'].isin(split1_block23) & x['buildingblock3_smiles'].isin(split1_block23)]

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
